# USING SPARK SQL IN PYTHON

```python
from pyspark.sql import SQLContext, Row
hiveContext = HiveContext(sc)
inputData = spark.read.json(dataFile)
inputData.createOrReplaceTempView('myStructuredStuff')
myResultDataFrame = hiveContext.sql('select foo from bar order by foobar')

myResultDataFrame.show()
myResultDataFrame.select('someFieldName')
myResultDataFrame.filter(myResultDataFrame('someFieldName' > 200))
myResultDataFrame.groupBy(myResultDataFrame('someFieldName')).mean()
myResultDataFrame.rdd().map(mapperFunction)
```

## USER-DEFINED FUNCTIONS(UDF'S)

```python
from pyspark.sql.types import IntegerType
hiveCtx.registerFunction('square', lambda x: x*x, IntegerType())
df = hiveCtx.sql("select square('someNumericFiled') from tableName")
```

In [1]:
from pyspark.sql import SparkSession, Row
import collections

In [2]:
# Create as SparkSession 
spark = SparkSession.builder.appName('SparkSQL').getOrCreate()

In [5]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [9]:
data_schema = [StructField('ID', IntegerType(), True),
               StructField('name', StringType(), True),
               StructField('age', IntegerType(), True),
               StructField('num_friends', IntegerType(), True)]

In [10]:
final_struc = StructType(data_schema)

In [11]:
people = spark.read.csv("fakefriends.csv", sep=',', encoding='utf-8', schema=final_struc)

In [12]:
people.show()

+---+--------+---+-----------+
| ID|    name|age|num_friends|
+---+--------+---+-----------+
|  0|    Will| 33|        385|
|  1|Jean-Luc| 26|          2|
|  2|    Hugh| 55|        221|
|  3|  Deanna| 40|        465|
|  4|   Quark| 68|         21|
|  5|  Weyoun| 59|        318|
|  6|  Gowron| 37|        220|
|  7|    Will| 54|        307|
|  8|  Jadzia| 38|        380|
|  9|    Hugh| 27|        181|
| 10|     Odo| 53|        191|
| 11|     Ben| 57|        372|
| 12|   Keiko| 54|        253|
| 13|Jean-Luc| 56|        444|
| 14|    Hugh| 43|         49|
| 15|     Rom| 36|         49|
| 16|  Weyoun| 22|        323|
| 17|     Odo| 35|         13|
| 18|Jean-Luc| 45|        455|
| 19|  Geordi| 60|        246|
+---+--------+---+-----------+
only showing top 20 rows



In [13]:
people.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- num_friends: integer (nullable = true)



In [16]:
people.createOrReplaceTempView("view_people")

In [17]:
# SQL can be run over DataFrames that have been registerd as a table
teenagers = spark.sql('SELECT * FROM view_people WHERE age >= 13 AND age <= 19')

In [18]:
teenagers.show()

+---+-------+---+-----------+
| ID|   name|age|num_friends|
+---+-------+---+-----------+
| 21|  Miles| 19|        268|
| 52|Beverly| 19|        269|
| 54|  Brunt| 19|          5|
|106|Beverly| 18|        499|
|115|  Dukat| 18|        397|
|133|  Quark| 19|        265|
|136|   Will| 19|        335|
|225|   Elim| 19|        106|
|304|   Will| 19|        404|
|341|   Data| 18|        326|
|366|  Keiko| 19|        119|
|373|  Quark| 19|        272|
|377|Beverly| 18|        418|
|404| Kasidy| 18|         24|
|409|    Nog| 19|        267|
|439|   Data| 18|        417|
|444|  Keiko| 18|        472|
|492|  Dukat| 19|         36|
|494| Kasidy| 18|        194|
+---+-------+---+-----------+



In [23]:
# We can also use functions instead of SQL queries
for teen in teenagers.collect():
    print(teen[0], teen[1], teen[2], teen[3])

21 Miles 19 268
52 Beverly 19 269
54 Brunt 19 5
106 Beverly 18 499
115 Dukat 18 397
133 Quark 19 265
136 Will 19 335
225 Elim 19 106
304 Will 19 404
341 Data 18 326
366 Keiko 19 119
373 Quark 19 272
377 Beverly 18 418
404 Kasidy 18 24
409 Nog 19 267
439 Data 18 417
444 Keiko 18 472
492 Dukat 19 36
494 Kasidy 18 194


In [25]:
# We can also use functions instead of SQL queries
teenagers.groupBy('age').count().orderBy('age').show()

+---+-----+
|age|count|
+---+-----+
| 18|    8|
| 19|   11|
+---+-----+



In [26]:
spark.stop()